# Tugas 7 : Query dengan kalimat

mencari 100 dokumen itu (masukkan 1 kalimat, ketemunya dokumen yg mana)

data tfidf ditransofrmasi menjadi sdikit dimensi menggunakan svd yg diambil v dan sigma (v tdk diambil smua)(Sebagian dri v) mentransformasi supaya dikit kolom

cosinus similtas (mirip mendekati 1) boleh pke euclidean

## Import Libraries

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive


## Mount Google Drive dan load data

In [ ]:
drive.mount('/content/drive')
data = pd.read_csv("/content/drive/My Drive/ppw/tugas/DataTugas2/data_terbaru1.csv")
data = data[['Judul Berita', 'Isi Berita', 'Tanggal Berita', 'Kategori Berita']]
data.head()

Mounted at /content/drive


,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
0,Pesan Anies ke Tom Lembong Tersangka Korupsi I...,Reporter\nNovali Panji Nugroho\nEditor\nEko Ar...,30-10-2024 05:55,Nasional
1,"Tom Lembong Tersangka Korupsi Impor Gula, Nama...",Reporter\nNovali Panji Nugroho\nEditor\nAhmad ...,29-10-2024 23:55,Nasional
2,"Profil Tom Lembong, Eks Mendag dan Co-Captain ...",Reporter\nNovali Panji Nugroho\nEditor\nAhmad ...,29-10-2024 22:55,Nasional
3,Tom Lembong Sempat Unggah Hal Ini Sehari Sebel...,Reporter\nTempo.co\nEditor\nAndry Triyanto Tji...,30-10-2024 02:55,Nasional
4,"Prabowo Ingin Tingkatkan Pembangunan di Papua,...",Reporter\nVedro Imanuel G\nEditor\nAgung Seday...,29-10-2024 23:55,Bisnis


## Drop Rows with Missing Values

- Menghapus baris yang tidak memiliki isi berita di kolom Isi Berita.
- Memastikan semua konten dalam kolom Isi Berita bertipe string.

In [ ]:
data = data.dropna(subset=['Isi Berita'])
data['Isi Berita'] = data['Isi Berita'].astype(str)
print(f"Jumlah data setelah menghapus baris kosong: {len(data)}")


Jumlah data setelah menghapus baris kosong: 99


<ipython-input-3-06d1254a6c14>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Isi Berita'] = data['Isi Berita'].astype(str)


## TF-IDF Vectorization

 Mengonversi teks Isi Berita menjadi matriks TF-IDF, yaitu representasi numerik berdasarkan bobot term-frequency.

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(data['Isi Berita'])
print("Dimensi TF-IDF Matrix:", tfidf_matrix.shape)

Dimensi TF-IDF Matrix: (99, 2163)


Output ini menunjukkan ukuran matriks setelah proses TF-IDF.

## Dimensionality Reduction with SVD

In [ ]:
svd = TruncatedSVD(n_components=50)
reduced_tfidf_matrix = svd.fit_transform(tfidf_matrix)
print("Dimensi Reduced TF-IDF Matrix:", reduced_tfidf_matrix.shape)

Dimensi Reduced TF-IDF Matrix: (99, 50)


Fungsi: Mengurangi dimensi dari matriks TF-IDF untuk meningkatkan efisiensi dengan TruncatedSVD, mengurangi dimensi menjadi 50 fitur.

## Query Input and Vectorization

In [ ]:
query = input("Masukkan kalimat untuk mencari dokumen terkait: ")
query_vector = tfidf_vectorizer.transform([query])
query_reduced = svd.transform(query_vector)

KeyboardInterrupt: Interrupted by user

## Calculate Cosine Similarity

Menghitung kesamaan antara query dan dokumen menggunakan cosine similarity.

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Misalkan query_reduced dan reduced_tfidf_matrix sudah didefinisikan
cos_similarities = cosine_similarity(query_reduced, reduced_tfidf_matrix).flatten()

# Bulatkan nilai cosine similarity hingga 3 digit setelah koma
cos_similarities_rounded = np.round(cos_similarities, decimals=3)

# Tampilkan 5 nilai teratas
print("Cosine Similarity :")
cos_similarities_rounded


## Sorting and Selecting Top Results

Mengambil 10 dokumen teratas berdasarkan nilai cosine similarity.

In [ ]:
top_100_indices = cos_similarities_rounded.argsort()[-100:][::-1]
top_100_docs = data.iloc[top_100_indices].copy()
top_100_docs['Cosine Similarity'] = cos_similarities_rounded[top_100_indices]

## Remove Duplicates and Display Results with Snippet

- Menghapus duplikat berdasarkan Judul Berita.
- Menyusun ulang tabel untuk menampilkan 100 kata pertama dari Isi Berita pada setiap hasil.
- Mengurutkan hasil berdasarkan Cosine Similarity dan menampilkan hasil akhir.

In [ ]:
import pandas as pd

# Sample data (assuming top_100_docs is already defined and processed)
top_100_docs = top_100_docs.drop_duplicates(subset='Judul Berita')
results_table = top_100_docs[['Judul Berita', 'Tanggal Berita', 'Kategori Berita', 'Cosine Similarity']].copy()

# Add a snippet of the news content
results_table['Isi Berita'] = top_100_docs['Isi Berita'].apply(lambda x: ' '.join(x.split()[:100]))

# Sort the results by Cosine Similarity
results_table = results_table.sort_values(by='Cosine Similarity', ascending=False)
results_table.reset_index(drop=True, inplace=True)

In [ ]:
results_table